---
title: Research of existing geojson files for Germany on municipality level
date: now
author: Jan Cap
---

We found multiple data sources of geojson files for Germany. First one on opendatalab.de: https://opendatalab.de/projects/geojson-utilities/#
and second one on https://public.opendatasoft.com/explore/assets/georef-germany-gemeinde/export/
Lets explore data sources and compare municipalities with each other and with municipality data from regionalstatistik.de.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from geoscore_de.data_flow.municipality import load_municipality_data

official_mun_df = load_municipality_data("../data/raw/municipalities_2022.csv")
official_mun_df.head(3)

## Opendatalab.de geojson

### Load data

Data from this source are not from official statistical office, so we will do some data comparison with official data.
Dataset contains a lot of metadata about each municipality, including following fields:
- RS (Regional key): 12-digit
- AGS (Official municipality key): 8-digit
- GEN (Geographical name): official name of the administrative unit
- BEZ (Official designation): official designation of the administrative unit like "Stadt", "Landkreis", etc.
- destatis (Destatis data): includes area, population numbers (men, women, total), density, zip codes, etc. in stringified dictionary format

In [ ]:
# Try to load municipality-level data (Gemeinden)
# Try to load German municipality boundaries from a common source

try:
    # Load GeoJSON data using geopandas
    gdf_mun = gpd.read_file("../data/gemeinden_simplify200.geojson")

    print(f"Loaded GeoDataFrame with {len(gdf_mun)} rows")
    print(f"Columns: {list(gdf_mun.columns)}")
    print(f"CRS: {gdf_mun.crs}")

    # Display first few rows
    print("\nFirst 3 rows:")
    display(gdf_mun.head(3))

except Exception as e:
    print(f"Error loading from URL: {e}")
    print("Let's try a different approach...")

In [ ]:
print("Rows with different RS and RS_0:", len(gdf_mun[gdf_mun["RS"] != gdf_mun["RS_0"]]))
print("Rows with different RS and SDV_RS:", len(gdf_mun[gdf_mun["RS"] != gdf_mun["SDV_RS"]]))
print("Rows with different AGS and AGS_0:", len(gdf_mun[gdf_mun["AGS"] != gdf_mun["AGS_0"]]))

RS_0, SDV_RS and AGS_0 are identical to RS and AGS columns. We can drop them.

In [ ]:
gdf_mun = gdf_mun.drop(columns=["RS_0", "SDV_RS", "AGS_0"])

In [ ]:
# explore Hamburg municipalities
gdf_mun[gdf_mun["AGS"].str.startswith("0200")]

In [ ]:
# Visualize the GeoJSON data
# Create a simple plot
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
gdf_mun.plot(ax=ax, color="lightblue", edgecolor="black", linewidth=0.3)
ax.set_title("German Municipalities (Gemeinden)")
ax.set_axis_off()
plt.tight_layout()
plt.show()

# Show some basic statistics
print("\nGeoDataFrame Info:")
print(f"Shape: {gdf_mun.shape}")
print(f"Geometry type: {gdf_mun.geometry.geom_type.unique()}")
print(f"Bounds: {gdf_mun.total_bounds}")

### Compare with official municipality data

In [ ]:
# merge on AGS and compare which municipalities are missing

merged_df = gdf_mun.merge(official_mun_df, on="AGS", how="outer", indicator=True)
merged_df["_merge"].value_counts()

There is 472 municipalities more in geojson file than in official data. Lets see which ones are missing.

In [ ]:
import matplotlib.patches as mpatches

# plot map with extra municipalities from geojson file highlighted
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
merged_df[merged_df["_merge"] == "both"].plot(
    ax=ax, color="lightblue", edgecolor="black", linewidth=0.3, label="In both"
)
merged_df[merged_df["_merge"] == "left_only"].plot(
    ax=ax, color="red", edgecolor="black", linewidth=0.3, label="Only in geojson"
)
ax.set_title("German Municipalities Comparison")
ax.set_axis_off()

# Manually create legend handles
handles = [
    mpatches.Patch(color="lightblue", label="In both"),
    mpatches.Patch(color="red", label="Only in Opendatalab geojson"),
]
ax.legend(handles=handles)

plt.tight_layout()
plt.show()

In [ ]:
merged_df[merged_df["_merge"] == "left_only"]

## opendatasoft.com geojson

### Load geojson

In [ ]:
import json

from shapely.geometry import shape

ods_gdf_mun = gpd.read_file("../data/georef-germany-gemeinde.csv")
ods_gdf_mun["geometry"] = ods_gdf_mun["Geo Shape"].apply(lambda x: shape(json.loads(x)) if pd.notnull(x) else None)
ods_gdf_mun = ods_gdf_mun.drop(columns=["Geo Shape"])
ods_gdf_mun = gpd.GeoDataFrame(ods_gdf_mun, geometry="geometry", crs="EPSG:4326")
ods_gdf_mun.head(3)

Each row has multiple IDs, but no AGS. We will need to calculate AGS from other IDs. Specifically from Gemeinde code, which contains AGS within itself.

In [ ]:
ods_gdf_mun["AGS"] = ods_gdf_mun["Gemeinde code"].str.slice(0, 5) + ods_gdf_mun["Gemeinde code"].str.slice(9, 12)
# see if the hamburg municipality has correct AGS
ods_gdf_mun[ods_gdf_mun["AGS"].str.startswith("0200")]

### Compare with official data

In [ ]:
ods_merged_df = ods_gdf_mun.merge(official_mun_df, on="AGS", how="outer", indicator=True)
ods_merged_df["_merge"].value_counts()

In [ ]:
import matplotlib.patches as mpatches

# plot map with extra municipalities from geojson file highlighted
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ods_merged_df[ods_merged_df["_merge"] == "both"].plot(
    ax=ax, color="lightblue", edgecolor="black", linewidth=0.3, label="In both"
)
ods_merged_df[ods_merged_df["_merge"] == "left_only"].plot(
    ax=ax, color="red", edgecolor="black", linewidth=0.3, label="Only in geojson"
)
ax.set_title("German Municipalities Comparison")
ax.set_axis_off()

# Manually create legend handles
handles = [
    mpatches.Patch(color="lightblue", label="In both"),
    mpatches.Patch(color="red", label="Only in opendatasoft geojson"),
]
ax.legend(handles=handles)

plt.tight_layout()
plt.show()

It has about half more municipalities compared to opendatalab.de geojson. Those municipalities seems to be on the same locations as the extra municipalities in opendatalab.de geojson.

In [ ]:
# sort by geometry size and show extra municipalities
ods_merged_df["geometry_area"] = ods_merged_df.geometry.area
ods_merged_df[ods_merged_df["_merge"] == "left_only"].sort_values(by="geometry_area", ascending=False)

In [ ]:
# filter all where Verwaltungsgemeinschaft code has 9 on 6.th place and show how many _merged are which value
ods_merged_df[ods_merged_df["Verwaltungsgemeinschaft code"].str[5] == "9"]["_merge"].value_counts()

In [ ]:
ods_merged_df[(ods_merged_df["Verwaltungsgemeinschaft code"].str[5] == "9") & (ods_merged_df["_merge"] == "both")]

We tried to find something about these extra municipalities. Some of them are nature parks or uninhabited areas. Also all of these extra municipalities have Verwaltungsgemeinschaft code with 9 on 6.th place (all other municipalities have 0 on that place). Possibly these are some kind of special municipalities used for administrative purposes only. There are 2 municipalities that are matched between geojson and official data and have Verwaltungsgemeinschaft code with 9 on 6.th place. These are "Gemeindefreies Gebiet Osterheide" and "Gemeindefreies Gebiet Lohheide". Both are mostly forrest areas with very low population.

We can drop all areas that were not matched between geojson and official data.

In [ ]:
ods_merged_df = ods_merged_df[ods_merged_df["_merge"] != "left_only"]

In [ ]:
# plot map with extra municipalities from geojson file highlighted
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ods_merged_df[ods_merged_df["_merge"] == "both"].plot(
    ax=ax, color="lightblue", edgecolor="black", linewidth=0.3, label="In both"
)
ax.set_title("German Municipalities (nature reservations dropped)")
ax.set_axis_off()


plt.tight_layout()
plt.show()

Now we should look into municipalities that are in official data but not in geojson.

In [ ]:
ods_merged_df[ods_merged_df["_merge"] == "right_only"]